In [24]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression


In [25]:
spark = SparkSession.builder.appName("Pipeline").getOrCreate()

In [6]:
# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))],
["label", "features"])

training.show(truncate=False)

+-----+--------------+
|label|features      |
+-----+--------------+
|1.0  |[0.0,1.1,0.1] |
|0.0  |[2.0,1.0,-1.0]|
|0.0  |[2.0,1.3,1.0] |
|1.0  |[0.0,1.2,-0.5]|
+-----+--------------+



In [10]:
# create a\n logistic regression instance
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [14]:
# lr.fit()
# print out the parameters, documantation, and any default value
print(f"LogisticRegression paramerts:-\n{lr.explainParams()} ")

LogisticRegression paramerts:-
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bou

In [15]:
model1 = lr.fit(training)
# 
model1

LogisticRegressionModel: uid=LogisticRegression_b59a7fa1d2eb, numClasses=2, numFeatures=3

In [16]:
model1.extractParamMap()

{Param(parent='LogisticRegression_b59a7fa1d2eb', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_b59a7fa1d2eb', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_b59a7fa1d2eb', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_b59a7fa1d2eb', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LogisticRegression_b59a7fa1d2eb', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_b59a7fa1d2eb', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LogisticRegression_b59a7fa1d2eb', name='maxBlockSizeInMB', doc='maximum memory in MB for s

In [17]:
 # We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
# Specify multiple Params.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # type: ignore

# You can combine paramMaps, which are python dictionaries.
# Change output column name
paramMap2 = {lr.probabilityCol: "myProbability"}
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)  # type: ignore

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())

Model 2 was fit using parameters: 
{Param(parent='LogisticRegression_b59a7fa1d2eb', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_b59a7fa1d2eb', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_b59a7fa1d2eb', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_b59a7fa1d2eb', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_b59a7fa1d2eb', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_b59a7fa1d2eb', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_b59a7fa1d2eb', name='maxBlockSizeInMB', do

In [18]:
  # Prepare test data
test = spark.createDataFrame([
        (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
        (0.0, Vectors.dense([3.0, 2.0, -0.1])),
        (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

In [19]:
 # Make predictions on test data using the Transformer.transform() method.
    # LogisticRegression.transform will only use the 'features' column.
    # Note that model2.transform() outputs a "myProbability" column instead of the usual
    # 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(test)
result = prediction.select("features", "label", "myProbability", "prediction") \
    .collect()

In [22]:
for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
            % (row.features, row.label, row.myProbability, row.prediction))
# $example off$

features=[-1.0,1.5,1.3], label=1.0 -> prob=[0.05707304171034015,0.9429269582896599], prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> prob=[0.9238522311704104,0.07614776882958962], prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> prob=[0.10972776114779423,0.8902722388522057], prediction=1.0


In [23]:
# spark.stop()

#### Pipeline Example

In [27]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [28]:
# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

training.show()

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



In [29]:
# configure a ML Pipeline,
tokenizer = Tokenizer(inputCol='text', outputCol='words')
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features')
lr = LogisticRegression(maxIter=10, regParam=0.001)

In [30]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [31]:
model = pipeline.fit(training)

In [32]:
# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

test.show()

+---+------------------+
| id|              text|
+---+------------------+
|  4|       spark i j k|
|  5|             l m n|
|  6|spark hadoop spark|
|  7|     apache hadoop|
+---+------------------+



In [33]:
prediction = model.transform(test)

In [36]:
prediction.select("id", "text", "probability", "prediction")

DataFrame[id: bigint, text: string, probability: vector, prediction: double]

In [37]:
prediction.show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[19036,68...|[-1.6609033227473...|[0.15964077387874...|       1.0|
|  5|             l m n|           [l, m, n]|(262144,[1303,526...|[1.64218895265635...|[0.83783256854766...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[173558,1...|[-2.5980142174392...|[0.06926633132976...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[68303,19...|[4.00817033336806...|[0.98215753334442...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+

